In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

################## Extract all data from excel File

xl = pd.ExcelFile("FoodFilterdata.xlsx")
#print(xl.sheet_names)

abbrev = xl.parse('ABBREV')
#abbrev = pd.read_excel(dffile, sheetname = 'ABBREV', header = 0, convert_float = True)
datasrc = xl.parse('DATA_SRC')
datsrcln = xl.parse('DATSRCLN')
derivcd = xl.parse('DERIV_CD')
fdgroup = xl.parse('FD_GROUP')
fooddes = xl.parse('FOOD_DES')
footnote = xl.parse('FOOTNOTE')
langdesc = xl.parse('LANGDESC')
langual = xl.parse('LANGUAL')
nutdata = xl.parse('NUT_DATA')
nutdef = xl.parse('NUTR_DEF')
srccd = xl.parse('SRC_CD')
weight = xl.parse('WEIGHT')

##################  Now extract RDA data
rdadata = pd.ExcelFile("RDA-DV for vitamins and fat.xlsx")
dv = rdadata.parse('DV')

#################  Join the necessary tables together into DF that contains all required data
abbrev_fddes = pd.merge(abbrev, fooddes, on='NDB_No', how='inner')
abbrev_foodgrp = pd.merge(abbrev_fddes, fdgroup, left_on='FdGrp_Cd',right_on='FdGrp_CD', how='inner')
abbrev_nutdata = pd.merge(abbrev_foodgrp, nutdata, left_on='NDB_No',right_on='NDB_No', how='inner')
allfooddata = pd.merge(abbrev_nutdata, nutdef, left_on='Nutr_No',right_on='Nutr_No', how='inner')
af = allfooddata


In [2]:
#print(rdadata.sheet_names)

##########   Rename dv Columns to adhere to proper naming convention and massage RDA data so we can work with it
dv.rename(columns = {'Food Component' : 'FoodComponent'}, inplace = True)
dv['DV'] = dv['DV'].str.replace("International Units","").str.replace("milligrams","").str.replace("micrograms","").str.replace("grams","").str.replace(",","").str.replace("(","").str.replace(")","")
#dv['DV'] = dv('DV').str.split(" ")
dv =dv.join(pd.DataFrame(dv.DV.str.split(' ',1).tolist(),columns=['Qty', 'Units']))
#Remove spaces from Units
dv['Units'] = dv['Units'].str.replace(' ',"")
# Add RDA Calories for males and females
Caloriesm = 2000
Caloriesf = 2000
#Change Qty to numeric 
dv.Qty = pd.to_numeric(dv.Qty)
#dv.dtypes

af = allfooddata
Searchstring = "Beverages|Fast Foods|CANDIES|Snacks|SYRUP"
df=af[af.FdGrp_Desc.str.contains(Searchstring) == False]
#dv.columns.tolist()

In [3]:
# Set up the multipliers for RDA for each FoodComponent 
Mult = []
for i in range(len(dv)) : 
    if dv.Units[i] == 'g' :
        mult = 1 * dv.Qty[i]
    elif dv.Units[i] == 'mg' :
        mult = 1/1000 * dv.Qty[i]
    elif dv.Units[i] == 'IU' :
        mult = 1 * dv.Qty[i]
    elif dv.Units[i] == 'µg' :
        mult = 1/1000000 * dv.Qty[i]
    else :
        print("Something went wrong in parsing g,mg,etc")
    
    Mult.append( mult)

dv['Mult']= Mult
#print(dv)
#### Formatting of the dv data frame complete

In [4]:
#####  Now it is time to format the larger df data frame

def renamecolumn(dataf, colfrom, colto) :
    dataf.rename(columns= {colfrom : colto}, inplace = True)

oldname={}
newname={}
oldname = df.columns.tolist()
newname = df.columns.str.replace("(","").str.replace(")","").str.replace(" ","")

for i in range(len(oldname)) : 
    #print("names " + oldname[i] + " " + newname[i] )
    renamecolumn(df, oldname[i],newname[i] )

/home/art/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [53]:
dv.head(10)

,FoodComponent,DV,Qty,Units,Mult
0,Total Fat,65 g,65.0,g,65.00
1,Saturated Fat,20 g,20.0,g,20.00
2,Cholesterol,300 mg,300.0,mg,0.30
3,Sodium,2400 mg,2400.0,mg,2.40
4,Potassium,3500 mg,3500.0,mg,3.50
5,Total Carbohydrate,300 g,300.0,g,300.00
6,Dietary Fiber,25 g,25.0,g,25.00
7,Protein,50 g,50.0,g,50.00
8,Vitamin A,5000 IU,5000.0,IU,5000.00
9,Vitamin C,60 mg,60.0,mg,0.06


In [54]:
df[0:10]

,NDB_No,Shrt_Desc_x,Water_g,Energ_Kcal,Protein_g,Lipid_Tot_g,Ash_g,Carbohydrt_g,Fiber_TD_g,Sugar_Tot_g,...,DF,Low_EB,Up_EB,Stat_Cmt,AddMod_Date,Units,Tagname,NutrDesc,Num_Dec,SR_Order
0,17144,"GAME MEAT,ANTELOPE,RAW",74.08,114,22.38,2.03,1.10,0.00,0.0,NaN,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
1,17145,"GAME MEAT,ANTELOPE,CKD,RSTD",65.90,150,29.45,2.67,1.45,0.00,0.0,NaN,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
2,17146,"GAME MEAT,BEAR,RAW",71.20,161,20.10,8.30,0.70,0.00,0.0,NaN,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
3,17147,"GAME MEAT,BEAR,CKD,SIMMRD",53.55,259,32.42,13.39,1.13,0.00,0.0,0.0,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
4,17148,"BISON,GROUND,GRASS-FED,CKD",65.09,179,25.45,8.62,1.02,0.00,0.0,0.0,...,8.0,24.306,26.604,4,03/2008,g,PROCNT,Protein,2,600
5,17149,"BISON,GROUND,GRASS-FED,RAW",71.59,146,20.23,7.21,0.91,0.05,0.0,0.0,...,8.0,19.545,20.924,4,03/2008,g,PROCNT,Protein,2,600
6,17150,"GAME MEAT,BEAVER,RAW",70.97,146,24.05,4.80,1.00,0.00,0.0,NaN,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
7,17151,"GAME MEAT,BEAVER,CKD,RSTD",57.92,212,34.85,6.96,1.45,0.00,0.0,0.0,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
8,17152,"GAME MEAT,BEEFALO,COMP OF CUTS,RAW",70.89,143,23.30,4.80,0.92,0.00,0.0,NaN,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600
9,17153,"GAME MEAT,BEEFALO,COMP OF CUTS,CKD,RSTD",61.70,188,30.66,6.32,1.21,0.00,0.0,NaN,...,NaN,NaN,NaN,NaN,04/1989,g,PROCNT,Protein,2,600


In [51]:
df.columns.tolist()

['NDB_No',
 'Shrt_Desc_x',
 'Water_g',
 'Energ_Kcal',
 'Protein_g',
 'Lipid_Tot_g',
 'Ash_g',
 'Carbohydrt_g',
 'Fiber_TD_g',
 'Sugar_Tot_g',
 'Calcium_mg',
 'Iron_mg',
 'Magnesium_mg',
 'Phosphorus_mg',
 'Potassium_mg',
 'Sodium_mg',
 'Zinc_mg',
 'Copper_mg',
 'Manganese_mg',
 'Selenium_µg',
 'Vit_C_mg',
 'Thiamin_mg',
 'Riboflavin_mg',
 'Niacin_mg',
 'Panto_Acid_mg',
 'Vit_B6_mg',
 'Folate_Tot_µg',
 'Folic_Acid_µg',
 'Food_Folate_µg',
 'Folate_DFE_µg',
 'Choline_Tot_mg',
 'Vit_B12_µg',
 'Vit_A_IU',
 'Vit_A_RAE',
 'Retinol_µg',
 'Alpha_Carot_µg',
 'Beta_Carot_µg',
 'Beta_Crypt_µg',
 'Lycopene_µg',
 'Lut+Zea_µg',
 'Vit_E_mg',
 'Vit_D_µg',
 'Vit_D_IU',
 'Vit_K_µg',
 'FA_Sat_g',
 'FA_Mono_g',
 'FA_Poly_g',
 'Cholestrl_mg',
 'GmWt_1',
 'GmWt_Desc1',
 'GmWt_2',
 'GmWt_Desc2',
 'Refuse_Pct',
 'FdGrp_Cd',
 'Long_Desc',
 'Shrt_Desc_y',
 'ComName',
 'ManufacName',
 'Survey',
 'Ref_Desc',
 'Refuse',
 'SciName',
 'N_Factor',
 'Pro_Factor',
 'Fat_Factor',
 'CHO_Factor',
 'FdGrp_CD',
 'FdGrp_Desc'

In [48]:
#dffin = df[np.isfinite(df['Healthy'])]
#Set up Bokeh
import bokeh.embed
import bokeh.io
import bokeh.models
import bokeh.models.widgets
import bokeh.plotting
import bokeh
from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider, HoverTool, BoxAnnotation, widgets
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from collections import OrderedDict
bokeh.plotting.output_notebook()

Loading BokehJS ...

In [49]:
savedv = dv


In [237]:
#dv.Qty[dv.FoodComponent == 'Protein'] 
#df[['Protein_(g)']]/
#pepe = dv.loc[dv.FoodComponent == 'Protein'].Qty * df['Protein_(g)']
#type(df[['Protein_(g)']]) 
dv.Qty[dv.FoodComponent == 'Protein']
#df['Protein_(g)'] * dv.Qty[dv.FoodComponent == 'Protein']
#df['Protein_g']/dv['DV']['Protein']
#pepe
df.dtypes

NDB_No                   int64
Shrt_Desc_x             object
Water_(g)              float64
Energ_Kcal               int64
Protein_(g)            float64
Lipid_Tot_(g)          float64
Ash_(g)                float64
Carbohydrt_(g)         float64
Fiber_TD_(g)           float64
Sugar_Tot_(g)          float64
Calcium_(mg)           float64
Iron_(mg)              float64
Magnesium_(mg)         float64
Phosphorus_(mg)        float64
Potassium_(mg)         float64
Sodium_(mg)            float64
Zinc_(mg)              float64
Copper_mg)             float64
Manganese_(mg)         float64
Selenium_(µg)          float64
Vit_C_(mg)             float64
Thiamin_(mg)           float64
Riboflavin_(mg)        float64
Niacin_(mg)            float64
Panto_Acid_mg)         float64
Vit_B6_(mg)            float64
Folate_Tot_(µg)        float64
Folic_Acid_(µg)        float64
Food_Folate_(µg)       float64
Folate_DFE_(µg)        float64
                        ...   
Calories               float64
Totfat_r

In [76]:
#for i in range(len(df)) :
    #value for value in x if not math.isnan(value)
 #   (df['Protein_g'])/(dv[dv.FoodComponent == 'Protein'].Mult) 
(df['Protein_g'])/(dv[dv.FoodComponent == 'Protein'].Mult ).dropna()
#df[np.isfinite(df['EPS'])]
    
    
    #df['Protein_rda']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7         0.697
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15          NaN
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
21          NaN
22          NaN
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
          ...  
679208      NaN
679209      NaN
679210      NaN
679211      NaN
679212      NaN
679213      NaN
679214      NaN
679215      NaN
679216      NaN
679217      NaN
679218      NaN
679219      NaN
679220      NaN
679221      NaN
679222      NaN
679223      NaN
679224      NaN
679225      NaN
679226      NaN
679227      NaN
679228      NaN
679229      NaN
679230      NaN
679231      NaN
679232      NaN
679233      NaN
679234      NaN
679235      NaN
679236      NaN
679237      NaN
Length: 610616, dtype: f

In [51]:
df['Protein_rda'] = df['Protein_g'])/dv.FoodComponent == 'Protein'] ) /100)
df['Calories_rda'] = df['Energ_Kcal']/Caloriesm * (df['GmWt_1']/100)
df['Calories'] = df['Energ_Kcal'] * (df['GmWt_1']/100)
df['Totfat_rda'] = df['Lipid_Tot_(g)']/dv.Qty[dv['Food Component']=='Total Fat']  * (df['GmWt_1']/100)
df['Satfat_rda'] = df['FA_Sat_(g)']/dv.Qty[dv['Food Component']=='Saturated Fat']  * (df['GmWt_1']/100)
df['Colesterol_rda'] = df['Cholestrl_(mg)']/dv.Qty[dv['Food Component']== 'Cholesterol']  * (df['GmWt_1']/100)
df['Potassium_rda'] = df['Potassium_(mg)']/dv.Qty[dv['Food Component']== 'Potassium'] * (df['GmWt_1']/100)
df['Sodium_rda'] = df['Sodium_(mg)']/dv.Qty[dv['Food Component']== 'Sodium']  * (df['GmWt_1']/100)
df['Dietfiber_rda'] = df['Fiber_TD_(g)']/dv.Qty[dv['Food Component']== 'Dietary Fiber']  * (df['GmWt_1']/100)
df['VitaminA_rda'] = df['Vit_A_IU']/dv.Qty[dv['Food Component']== 'Vitamin A']  * (df['GmWt_1']/100)
df['VitaminC_rda'] = df['Vit_C_(mg)']/dv.Qty[dv['Food Component']== 'Vitamin C'] * (df['GmWt_1']/100)
df['Calcium_rda'] = df['Calcium_(mg)']/dv.Qty[dv['Food Component']== 'Calcium']  * (df['GmWt_1']/100)
df['Iron_rda'] = df['Iron_(mg)']/dv.Qty[dv['Food Component']== 'Iron']  * (df['GmWt_1']/100)
df['VitaminD_rda'] = df['Vit_D_µg']/dv.Qty[dv['Food Component']== 'Vitamin D']  * (df['GmWt_1']/100)
df['VitaminE_rda'] = df['Vit_E_(mg)']/dv.Qty[dv['Food Component']== 'Vitamin E']  * (df['GmWt_1']/100)
df['VitaminK_rda'] = df['Vit_K_(µg)']/dv.Qty[dv['Food Component']== 'Vitamin K']  * (df['GmWt_1']/100)
df['Thiamin_rda'] = df['Thiamin_(mg)']/dv.Qty[dv['Food Component']== 'Thiamin']  * (df['GmWt_1']/100)
df['Riboflavin_rda'] = df['Riboflavin_(mg)']/dv.Qty[dv['Food Component']== 'Riboflavin']  * (df['GmWt_1']/100)
df['Niacin_rda'] = df['Niacin_(mg)']/dv.Qty[dv['Food Component']== 'Niacin']  * (df['GmWt_1']/100)
df['VitaminB6_rda'] = df['Vit_B6_(mg)']/dv.Qty[dv['Food Component']== 'Vitamin B6']  * (df['GmWt_1']/100)
df['Folate_rda'] = df['Folate_Tot_(µg)']/dv.Qty[dv['Food Component']== 'Folate']  * (df['GmWt_1']/100)
df['VitaminB12_rda'] = df['Vit_B12_(µg)']/dv.Qty[dv['Food Component']== 'Vitamin B12']  * (df['GmWt_1']/100)
df['PantothenicAcid_rda'] = df['Panto_Acid_mg)']/dv.Qty[dv['Food Component']== 'Pantothenic acid']  * (df['GmWt_1']/100)
df['Phosphorus_rda'] = df['Phosphorus_(mg)']/dv.Qty[dv['Food Component']== 'Phosphorus']  * (df['GmWt_1']/100)
df['Magnesium_rda'] = df['Magnesium_(mg)']/dv.Qty[dv['Food Component']== 'Magnesium']  * (df['GmWt_1']/100)
df['Zinc_rda'] = df['Zinc_(mg)']/dv.Qty[dv['Food Component']== 'Zinc']  * (df['GmWt_1']/100)
df['Selenium_rda'] = df['Selenium_(µg)']/dv.Qty[dv['Food Component']== 'Selenium']  * (df['GmWt_1']/100)
df['Copper_rda'] = df['Copper_mg)']/dv.Qty[dv['Food Component']== 'Copper']  * (df['GmWt_1']/100)
df['Manganese_rda'] = df['Manganese_(mg)']/dv.Qty[dv['Food Component']== 'Manganese']  * (df['GmWt_1']/100)
df['Molybdenum_rda'] = df['Energ_Kcal']/dv.Qty[dv['Food Component']== 'Molybdenum']  * (df['GmWt_1']/100)

df['Healthy'] = df['Protein_rda']+ df['Dietfiber_rda']+ df['Thiamin_rda'] + df['Riboflavin_rda'] + df['Niacin_rda']  + df['PantothenicAcid_rda'] + df['Phosphorus_rda'] + df['Magnesium_rda'] + df['Selenium_rda'] + df['Copper_rda'] + df['Manganese_rda'] + df['Molybdenum_rda']  + (df['VitaminA_rda'] + df['VitaminB6_rda'] + df['VitaminC_rda'] + df['Calcium_rda'] + df['Iron_rda'] + df['VitaminD_rda'] + df['VitaminE_rda'] + df['VitaminK_rda'] + df['Folate_rda'] + df['VitaminB12_rda'] + df['Zinc_rda'])*2 - (10*df['Sodium_rda']) 

# df["Quantity"] = df["GmWt_Desc1"] 
#df.columns.tolist()


AttributeError: module 'pandas' has no attribute 'tonumeric'

In [50]:
 #df['Protein_(g)']/dv.Qty[dv['Food Component']=='Protein'] * (df['GmWt_1']/100)
type(df['Protein_(g)'])

pandas.core.series.Series

In [14]:
dffin = df
dffin = dffin.sort_values('Nutr_Val')
smalldfin = dffin.head(500)

## Modifying Max calories  to fit my whims
## will loop in nutritionist later
Maxgoodcal = 400
Redboxhigh = 600
Yellowboxlow = 400
ds1={}

# Setup the plot
high_box = BoxAnnotation( left=Redboxhigh, fill_alpha=0.1, fill_color='red')
mid_box = BoxAnnotation( left=Yellowboxlow, right=Redboxhigh, fill_alpha=0.1, fill_color='yellow')
low_box = BoxAnnotation( right=Yellowboxlow, fill_alpha=0.1, fill_color='green')


TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,previewsave,box_select,poly_select,lasso_select,hover"
plot = figure(title= "Nutrition Filter Test",tools = TOOLS)
#plot.scatter('x', 'y', source=tinyds1, radius= dffin.Healthy ,line_color=None)
plot.scatter('x', 'y', source=ds1, radius= 50 ,line_color=None)
plot.add_layout(low_box)
plot.add_layout(mid_box)
plot.add_layout(high_box)
#plot.title.text = "Nutrition Filter"
#p.xgrid[0].grid_line_color=None
#p.ygrid[0].grid_line_alpha=0.5
plot.xaxis.axis_label = 'Calories(lower tends to be better)'
plot.yaxis.axis_label = 'Healthy (Higher is Better)'



x= dffin.Calories.values.tolist()
h= dffin.Nutr_Val.values.tolist()
y= dffin.Nutr_Val.values.tolist()
#n=dffin.Shrt_Desc.values.tolist()
n=dffin.Shrt_Desc_x.values.tolist()
s=dffin['Sodium_(mg)'].values.tolist()
k=dffin['Vit_K_(µg)'].values.tolist()
phos=dffin['Phosphorus_(mg)'].values.tolist()
#por=dffin.Quantity.values.tolist()
pro=dffin['Protein_(g)'].values.tolist()
pot=dffin['Potassium_(mg)'].values.tolist()

#ylim = max(h)+50
ds1 = ColumnDataSource(data=dict(x=x,y=y,n=n,s=s,h=h,pro=pro,phos=phos,k=k, pot=pot))




In [15]:

ds1.callback = CustomJS(args=dict(source=ds1), code="""
    var data = source.data;
    var S = salt.value;
    var Prot = protein.value;
    var Phos = phosphorus.value;
    var K = vit_k.value;
    var Pot = pot.value;
    
    h = data['h']
    x = data['x']
    y = data['y']
    s = data['s']
    k = data['k']
    pot = data['pot']
    pro = data['pro']
    phos = data['phos']
    por = data['por']
    


for (i = 0; i < x.length; i++) {
        y[i] =  h[i]-((s[i]-100)*S)- ((pro[i]- 10)*Prot *3) - ((phos[i]- 100)*Phos *2) - ((k[i] - 200)* K *1 ) - ((pot[i] - 200)* Pot *1 );
        
        if (y[i] < 0) {
            y[i] = 0
            }   
        }
#  text_input = TextInput(title="Filter", value='', callback = text_input_callback)

#plot.title.text = "Food filter Jan 20 Snapshot"      
    
    source.trigger('change');
""")


salt_slider = Slider(start=0, end=1, value=0, step=.005, title="Salt", callback=ds1.callback)
ds1.callback.args["salt"] = salt_slider

protein_slider = Slider(start=0, end=1, value=0, step=.005, title="Protein", callback=ds1.callback)
ds1.callback.args["protein"] = protein_slider

phos_slider = Slider(start=0, end=1, value=0, step=.005, title="Phosphorus", callback=ds1.callback)
ds1.callback.args["phosphorus"] = phos_slider

k_slider = Slider(start=0, end=1, value=0, step=.005, title="Vitamin K", callback=ds1.callback)
ds1.callback.args["vit_k"] = k_slider

pot_slider = Slider(start=0, end=1, value=0, step=.005, title="Potassium", callback=ds1.callback)
ds1.callback.args["pot"] = pot_slider

hover = plot.select(dict(type=HoverTool))
hover.tooltips = OrderedDict([('Food', '@n'),('Nutrition', '@y'),('Sodium', '@s'),('Phosphorus', '@phos'),('Calories', '@x'),('Protein', '@pro'),('Potassium', '@pot'),('Vitamin K', '@k'),('Quantity','@por')])

layout = row(
     plot,
    widgetbox(salt_slider,protein_slider, phos_slider, k_slider, pot_slider),
)
#
    
output_file("ffslidertext.html", title="Food Filters")


show(layout)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:
# #original Code 
# df['protein_rda'] = df['Protein_g']/dv['DV']['Protein'] * (df['GmWt_1']/100)
# df['Calories_rda'] = df['Energ_Kcal']/dv['DV']['Caloriesm'] * (df['GmWt_1']/100)
# df['Calories'] = df['Energ_Kcal'] * (df['GmWt_1']/100)
# #df['foodtext'] = df.Shrt_Desc 
# df['Totfat_rda'] = df['Lipid_Tot_g']/dv['DV']['Total Fat']  * (df['GmWt_1']/100)
# df['Satfat_rda'] = df['FA_Sat_g']/dv['DV']['Saturated Fat']  * (df['GmWt_1']/100)
# df['Colesterol_rda'] = df['Cholestrl_mg']/dv['DV']['Cholesterol']  * (df['GmWt_1']/100)
# df['Potassium_rda'] = df['Potassium_mg']/dv['DV']['Potassium'] * (df['GmWt_1']/100)
# df['Sodium_rda'] = df['Sodium_mg']/dv['DV']['Sodium']  * (df['GmWt_1']/100)
# #df['TotCarbs_rda'] = df['Lipid_Tot_g']/dv['DV']['Total Carbohydrate']  * (df['GmWt_1']/100)
# df['Dietfiber_rda'] = df['Fiber_TD_g']/dv['DV']['Dietary Fiber']  * (df['GmWt_1']/100)
# df['VitaminA_rda'] = df['Vit_A_IU']/dv['DV']['Vitamin A']  * (df['GmWt_1']/100)
# df['VitaminC_rda'] = df['Vit_C_mg']/dv['DV']['Vitamin C'] * (df['GmWt_1']/100)
# df['Calcium_rda'] = df['Calcium_mg']/dv['DV']['Calcium']  * (df['GmWt_1']/100)
# df['Iron_rda'] = df['Iron_mg']/dv['DV']['Iron']  * (df['GmWt_1']/100)
# df['VitaminD_rda'] = df['Vit_D_pg']/dv['DV']['Vitamin D']  * (df['GmWt_1']/100)
# df['VitaminE_rda'] = df['Vit_E_mg']/dv['DV']['Vitamin E']  * (df['GmWt_1']/100)
# df['VitaminK_rda'] = df['Vit_K_pg']/dv['DV']['Vitamin K']  * (df['GmWt_1']/100)
# df['Thiamin_rda'] = df['Thiamin_mg']/dv['DV']['Thiamin']  * (df['GmWt_1']/100)
# df['Riboflavin_rda'] = df['Riboflavin_mg']/dv['DV']['Riboflavin']  * (df['GmWt_1']/100)
# df['Niacin_rda'] = df['Niacin_mg']/dv['DV']['Niacin']  * (df['GmWt_1']/100)
# df['VitaminB6_rda'] = df['Vit_B6_mg']/dv['DV']['Vitamin B6']  * (df['GmWt_1']/100)
# df['Folate_rda'] = df['Folate_Tot_pg']/dv['DV']['Folate']  * (df['GmWt_1']/100)
# df['VitaminB12_rda'] = df['Vit_B12_pg']/dv['DV']['Vitamin B12']  * (df['GmWt_1']/100)
# df['PantothenicAcid_rda'] = df['Panto_Acid_mg']/dv['DV']['Pantothenic acid']  * (df['GmWt_1']/100)
# df['Phosphorus_rda'] = df['Phosphorus_mg']/dv['DV']['Phosphorus']  * (df['GmWt_1']/100)
# df['Magnesium_rda'] = df['Magnesium_mg']/dv['DV']['Magnesium']  * (df['GmWt_1']/100)
# df['Zinc_rda'] = df['Zinc_mg']/dv['DV']['Zinc']  * (df['GmWt_1']/100)
# df['Selenium_rda'] = df['Selenium_pg']/dv['DV']['Selenium']  * (df['GmWt_1']/100)
# df['Copper_rda'] = df['Copper_mg']/dv['DV']['Copper']  * (df['GmWt_1']/100)
# df['Manganese_rda'] = df['Manganese_mg']/dv['DV']['Manganese']  * (df['GmWt_1']/100)
# df['Molybdenum_rda'] = df['Energ_Kcal']/dv['DV']['Molybdenum']  * (df['GmWt_1']/100)
# ##df["Quantity"] = df["GmWt_1"].map(str) + " @ " + df["GmWt_Desc1"] 
# df["Quantity"] = df["GmWt_Desc1"] 
 
# df['Healthy'] = df['protein_rda']+ df['Dietfiber_rda']+ df['Thiamin_rda'] + df['Riboflavin_rda'] + df['Niacin_rda']  + df['PantothenicAcid_rda'] + df['Phosphorus_rda'] + df['Magnesium_rda'] + df['Selenium_rda'] + df['Copper_rda'] + df['Manganese_rda'] + df['Molybdenum_rda']  + (df['VitaminA_rda'] + df['VitaminB6_rda'] + df['VitaminC_rda'] + df['Calcium_rda'] + df['Iron_rda'] + df['VitaminD_rda'] + df['VitaminE_rda'] + df['VitaminK_rda'] + df['Folate_rda'] + df['VitaminB12_rda'] + df['Zinc_rda'])*2 - (10*df['Sodium_rda']) 

